<span class='note'><i>Make me look good.</i> Click on the cell below and press <kbd>Ctrl</kbd>+<kbd>Enter</kbd>.</span>

In [ ]:
from IPython.core.display import HTML
HTML(open('css/custom.css', 'r').read())

<h5 class='prehead'>SM286D &middot; Introduction to Applied Mathematics with Python &middot; Spring 2020 &middot; Uhan</h5>

<h5 class='lesson'>Lesson 17.</h5>

<h1 class='lesson_title'>Genetic algorithms with Python</h1>

## This lesson...

- Implementing a __genetic algorithm__ in Python.

---

## What is a genetic algorithm?

A __genetic algorithm (GA)__ is a type of algorithm for optimization problems, inspired by evolutionary biology. A GA tries to mimic the evolutionary process:

 - we start with a population of parents, 
 - we mate certain parents to produce a child, 
 - the child develops into a fully fledged adult, and 
 - the child may replace a less fit parent in the collection of parents. 
 - This process proceeds for many generations and ends with at least one very fit parent in the collection of parents. 

How does this work for an optimization problem? We want to find a solution that minimizes a certain objective function or __fitness function__. Each __parent__ will correspond to a solution to our optimization problem. The value of the fitness function for the parent is the parent's __score__. 

We start a generation by pairing up the parents. Each pair of parents will __mate__ to produce a new solution to our optimization problem, which we call a __child__. We "grow" the child into an adult by repeatedly modifying the child solution slightly and keeping the change if the child's score gets better. 

After a set number of such __local modifications__, we compare the child to its parents and replace one parent with the child if the child's score is better; otherwise we discard the child. A generation ends when each child has entered the parent pool or been discarded. We repeat the process for a set number of generations. 

This process involves many choices: we choose how the parent solutions combine to make a child input, we choose the fitness function, we choose the number of local modifications to the child, the types of local modifications allowed, the number of generations, and the number of parents. This approach is very flexible 
and not particularly tied to biology: one can incorporate or ignore the sex of parents, you could require three (or just one) parent to produce a child, etc. 

Note that in general, genetic algorithms are __heuristics__: they produce solutions that are only approximately optimal, usually without any guarantee of how close they are to optimal. This seems unsatsifying, but for very hard optimization problems, this might be the best we can do.

## The shortest path problem

We're going to create a genetic algorithm that finds the shortest path between two particular vertices in a graph. We've chosen this problem because (1) it is relatively easy to understand and (2) there exist fast exact algorithms so we can assess the quality of the solutions produced by our genetic algorithm.

We start with a graph $G$ with $n$ vertices or nodes. We assume that each vertex is connected to _all_ the other vertices by an edge.  This is known as a __complete graph__. 

The distances along each edge $(i,j)$ are stored in an $n \times n$ matrix $d$. We assume that $d_{ii} = 0$ for all $i$: the distance 
from a vertex to itself is zero. In addition, we assume that $d_{ij} = d_{ji} > 0$ if $i\neq j$: the distance for an edge is the same in both directions, and all distances between distinct nodes are positive. 

We fix a source node $s$ and a terminal node $t$. 
An __$s$-$t$ path__ is a sequence of vertices starting with the source node $s$ and ending at the terminal node $t$.
The __total distance__ of an $s$-$t$ path is the sum of the distances along each edge in the path.
We want to find the $s$-$t$ path with the lowest total distance.

## A genetic algorithm for the shortest path problem

How can we set up a genetic algorithm for the shortest path problem?

### The fitness function and population

We define the fitness function to take as input an $s$-$t$ path, and return the total distance of the path. Each parent (and child) corresponds to an $s$-$t$ path and its score. We'll start with a population of, say, 10 parents, each representing randomly generated $s$-$t$ paths. 

### Mating

In each generation, we'll form 5 pairs of parents, using every parent once. Given two parents, $P_0$ and $P_1$, we create a child in the following way. 

_Note._ We will often speak about parents and children as paths. This isn't strictly correct, because a parent or a child is both a path AND a score. We kindly ask for the reader's tolerance. 😊

**Case A.**
If $P_0$ and $P_1$ do not have a common vertex other than $s$ and $t$, then we randomly select a vertex $a$ in $P_0$ and a vertex $b$ in $P_1$. We form the child's path by joining the part of $P_0$ from $s$ to $a$, the edge $(a,b)$, and the part of $P_1$ from $b$ to $t$.


**Case B.**
If $P_0$ and $P_1$ have a common vertex other than $s$ and $t$, then let $x$ be the first vertex in $P_0$ that is in $P_1$. We form the child's path by doing one of the following: 

- join the part of the path $P_0$ from $s$ to $x$ with the part of the path $P_1$ from $x$ to $t$, or
- join the part of the path $P_1$ from $s$ to $x$ with the part of the path $P_0$ from $x$ to $t$.

We choose the path whose first part (up until $x$) has the smaller score. 

In both cases the new child's path may have loops. For instance if $s=0$ and $t=29$ we might produce a new child's path $0-5-6-9-5-9-19-29$. We remove the loop $5-6-9-5$ starting earliest in the path to get $0-5-9-19-29$. Note that this destroyed a second (entwined) loop $9-5-9$. In general, loops could remain in the resulting path. We continue to remove the earliest loops until none remain, producing our child's path. 

### Local modifications (a.k.a. "growing up")

For local modifications on a child, we pick two vertices $a$ and $b$ on the child's path, such that $a$ appears earlier than $b$. Then we consider three new candidate paths:

- the original child's path, 
- the path formed by joining
    1. the part of the child's path from $s$ to $a$, 
    2. the edge $(a,b)$, and 
    3. the part of the child's path from $b$ to $t$, and 
- the path formed by removing loops from the path obtained by joining:
    1. the part of the child's path from $s$ to $a$,
    2. two edge path from $a$ to a random vertex $c$ to $b$ 
    3. the part of the child's path from $b$ to $t$. 

We replace the child's path with the candidate path whose score is lowest. This completes a single local modification of the child. We do a set number of such modifications, say, 100. 

### Survival of the fittest

After each child grows up, they compete with their two parents. The two paths with the smallest scores remain in the mating pool and the path with the highest score is discarded. This completes a single generation. 

We repeat this entire process (pairing parents, mating to produce a child, local improvement of the child, and a fight-to-the-death elimination match pitting two parents against their child &mdash; hey, evolution isn't pretty!) over multiple generations, say, 50 times. Then we select the parent with the lowest score and use their path as our solution to the shortest path problem. 

### Is this algorithm good?

Note that our algorithm involves randomization, so running it multiple times will likely produce different solutions. Our algorithm is also a heuristic: it need not produce the shortest path but rather produces the shortest path out of the few paths it considered. 

Using the algorithm above, we consider $10+5*3*100*50$ (=75010) $s$-$t$ paths without loops. That is a lot of paths but if $n=30$ there are more than $8.28 \times 10^{29}$ $s$-$t$ paths, so we are only considering a  small subset of all paths. It is remarkable that this method often produces the best answer. 

We can check our answer using the `shortest_path` method in NetworkX, which solves the shortest path problem exactly. Exact algorithms for the shortest path problem, like Dijkstra's algorithm and the Bellman-Ford algorithm, are often taught in SA405 &mdash; Advanced Mathematical Programming.    

## Classwork

This lesson's classwork will walk you through implementing the genetic algorithm described above for the shortest path problem!

Before starting the classwork, run the code cell below to import NumPy.

In [ ]:
import numpy as np

**Problem 1.**
Write a function `make_distance_matrix` that 

- takes as input a positive integer $n$, and 
- returns an $n \times n$ symmetric matrix with 
    1. 0s for all diagonal entries and
    2. randomly generated integers between 1 and 99 (inclusive) for all off-diagonal entries. 

_Hints._

- Start with a $n \times n$ matrix filled with zeros.
- Use a nested `for` loop to go over rows with index $i = 0, 1, \dots, n-1$ and columns with index $j = i + 1, \dots, n - 1$. This will go over only the upper triangular entries of the matrix. Set $d_{ji} = d_{ij}$ so you end up with a symmetric matrix.
- Use the NumPy method `np.random.randint()`: `np.random.randint(a, b)` generates a random integer between `a` and `b - 1` inclusive. 

**Problem 2.**
Run the code cell beow.

The code sets some parameters, and calls your `make_distance_matrix` function from Problem 1 to create the matrix of distances between vertices in the graph.

In [ ]:
# Number of vertices
n = 30

# Source node
s = 0

# Terminal node
t = n - 1

# Number of parents
# Should be even
num_parents = 10

# Number of generations
num_gens = 50

# Number of local improvements
num_loc_improvement = 100

# Generate distance matrix
d = make_distance_matrix(n)

# Print distance matrix to check our work
print(d)

**Problem 3.**
Write a function `compute_score` that computes the total distance along a path.

- The inputs to this function should be
    1. a path starting at the source node ($s=0$) and ending at the terminal node ($t=29$) and 
    2. a distance matrix $d$.

- The path should be represented by a list of nodes, e.g. `[0, 5, 8, 12, 29]`
    
- This function should return the distance from the source node to the terminal node.

In [ ]:
# Write your code below


Check your function by running the code in the cell below.  If your function is correct, the code should output `True` to the screen.

In [ ]:
# Check your compute_score function by running the code in this cell
compute_score([0, 5, 8, 12, 29], d) == d[0, 5] + d[5, 8] + d[8, 12] + d[12, 29]

**Problem 4.**
Next, we will generate a list of parents. The code below defines a function `generate_parents`. Read through the code to get a sense of what it does.

In [ ]:
def generate_parents(num_parents, s, t, num_nodes, d):
    """
    Make a list of parents.
    Each parent consists of (1) a path from s to t, and (2) its score
    """
    # Initialize list of parents
    parents = []
    
    for i in range(num_parents):
        # Randomly generate a path p with the following steps
        # 1. Create a list of all the nodes
        nodes_except_s = list(range(num_nodes))
        
        # 2. Remove the source node from this list
        nodes_except_s.remove(s)
        
        # 3. Randomly shuffle the nodes in this list
        shuffled_nodes_except_s = list(np.random.permutation(nodes_except_s))
        
        # 4. Where does the terminal vertex appear in the shuffled list?
        t_index = shuffled_nodes_except_s.index(t)
        
        # 5. Create a random path by starting at the source node s,
        # continue with the nodes in the shuffled list, until we 
        # reach the terminal node t
        # Note that the + operator concatenates lists
        path = [s] + shuffled_nodes_except_s[:t_index + 1]
        
        # Compute the score of this random path
        score = compute_score(path, d)
        
        # Append this newly generated parent to the list of parents
        # Note that a parent is a dictionary
        parents.append({'path': path, 'score': score})
        
    return parents

In the next code cell, call the function `generate_parents`, and then print out a list of parents to the screen.

In [ ]:
# Write your code below


**Problem 5.**
Now we'll pair the parents for mating.

- Write code that creates a list `pairs` consisting of `num_parents / 2` inner lists. Each inner list should contain two parents. 
- Each parent should appear in one and only one list. 
- Your pairings should be random; you don't want to keep mating the same pair of parents generation after generation.

_Hint._ You might find the method `np.random.permutation()` useful. We used it in the definition of `generate_parents` in the previous problem. What does it do? Check the NumPy documentation if you're not sure.

__Problem 6.__
The code cell below defines the function `remove_loops`, which removes any loops in a path. Read through the code to get a sense of what it does.

In [ ]:
def remove_loops(p):
    """
    Remove loops from a path
    """
    
    # Keep doing this loop while there are 
    # no duplicates in the path (list of nodes) p.
    # The while condition uses the *set* data structure, which is
    # like a list, but requires that its items are unique.
    # list(set(x)) returns a list with all duplicates of x removed.
    while len(list(set(p))) != len(p):
        
        for i in list(set(p)):
            # Find the index of the first i 
            a = p.index(i) 
            
            # Find the index of the last i
            # Remember that p[::-1] is the list p, but in reverse order
            b = len(p) - 1 - p[::-1].index(i)
            
            # If the indices of the first and last i are
            # not the same, then we have a loop!
            if a != b:
                # Remove the vertices of the loop
                p = p[0: a + 1] + p[b + 1:]
                break
                
    return p

Try it out:

In [ ]:
remove_loops([0, 1, 2, 3, 5, 6, 3, 8, 5, 28, 21, 17, 29])

You should get `[0, 1, 2, 3, 8, 5, 28, 21, 17, 29]` as output.

Note that when we remove the loop starting at 3, the loop starting at 5 disappears! Removing the loop starting at 5 would kill the loop starting at 3 too. It isn't clear which is preferable. We just get a loopless path and call it a day.

The function `remove_loops` is called in the function `mate_parents`, which is defined in the code cell below. The function `mate_parents` takes as input two parents and the distance matrix $d$. Read through the code for `mate_parents` to get a sense of what it does.

In [ ]:
def mate_parents(p0, p1, d):
    """
    Produce a child by "mating" two parents p0 and p1
    """
    # Find the first vertex x after s in p0 that is in p1.
    # Such a vertex exists, since the terminal node t is in both p0 and p1.
    # (In this case, the loop below would go through all values of i.)
    for i in range(1, len(p0['path'])):
        if p0['path'][i] in p1['path']:
            break
            
    # After the loop above, i is the index of x in p0
    x = p0['path'][i]
        
    # CASE A: x = t
    # Select vertex a in p0 and vertex b in p1. 
    # Child = [s,...,a] in p0 + edge [a,b] + [b,...,t] in p1. 
    if x == p0['path'][-1]:
        
        # CASE A.1: both p0 and p1 have AT LEAST 
        # one vertex other than s and t in their paths
        # (i.e., they both have length > 2)
        if len(p0['path']) > 2 and len(p1['path']) > 2:
            # Randomly select vertex a in p0 by choosing a random index
            # (excluding the 0th and last indices, since they correspond
            # to s and t)
            a_index = np.random.randint(1, len(p0['path']) - 1)
            
            # Randomly select vertex b in p1 by choosing a random index
            b_index = np.random.randint(1, len(p1['path']) - 1)
            
            # Create child path: [s,...a] in p0 + [b,...,t] in p1
            child_path = p0['path'][:a_index + 1] + p1['path'][b_index:]
        
        # CASE A.2: p0 has at least one vertex other than s and t in its path,
        # but p1 has only s and t in its path
        if len(p0['path']) > 2 and len(p1['path']) == 2:
            # Randomly select vertex a in p0 by choosing a random index
            a_index = np.random.randint(1, len(p0['path']) - 1)
            
            # Select the terminal node from p1
            b_index = p1['path'][-1]
            
            # Create child path: [s,...,a] in p0 + [t]
            child_path = p0['path'][:a_index + 1] + p1['path'][b_index:]
        
        # CASE A.3: p0 has only s and t in its path,
        # but p1 has at least one vertex other than s and t in its path
        if len(p0['path']) == 2 and len(p1['path']) > 2:
            # Select the source node from p0
            a_index = p0['path'][0]
            
            # Randomly select vertex b in p1 by choosing a random index
            b_index = np.random.randint(1, len(p1['path']) - 1)
            
            # Create child path: [s] + [b,...,t] in p1
            child_path = p0['path'][:a_index + 1] + p1['path'][b_index:]
        
        # CASE A.4: p0 and p1 BOTH only have s and t in their paths
        else:
            # Set the child path to [s, t]
            child_path = p0['path']
        
    
    # CASE B: x != t
    # Select shorter of [s,...,x] on p0 and [s,...,x] on p1.
    # Add [x,...,t] from the other path.  
    else:
        # Find index of x in p1
        j = p1['path'].index(x)
        
        # Find the shorter [s,...,x]
        # Add [x,...,t] from the other path
        if compute_score(p0['path'][:i + 1], d) <= compute_score(p1['path'][:j + 1], d):
            child_path = p0['path'][:i + 1] + p1['path'][j + 1:]
        else: 
            child_path = p1['path'][:j + 1] + p0['path'][i + 1:]
            
    # Remove loops in the child path
    child_path = remove_loops(child_path)
    
    # Get child's score
    child_score = compute_score(child_path, d)
    
    # Return child
    child = {'path': child_path, 'score': child_score}
    return child

When looking at new code, it is a good idea to try to see what the code does to some sample input. The code below defines 3 parents, `parent0`, `parent1`, and `parent2`. 

Mate `parent0` and `parent1` to produce a child. Go line by line through the code for `mate_parents` above and predict what the function does. Then run the code to see if the child you predicted was correct.

Do the same with `parent0` and `parent2`. Here the function will return an answer with some randomness. What is different in this case? Why can't we confidently predict the path of the child?

In [ ]:
# Define parents for testing
parent0_path = [0, 5, 8, 12, 21, 29]
parent0_score = compute_score(parent0_path, d)
parent0 = {'path': parent0_path, 'score': parent0_score}

parent1_path = [0, 2, 8, 10, 15, 29]
parent1_score = compute_score(parent1_path, d)
parent1 = {'path': parent1_path, 'score': parent1_score}

parent2_path = [0, 7, 18, 29]
parent2_score = compute_score(parent2_path, d)
parent2 = {'path': parent2_path, 'score': parent2_score}

# Write your testing code below


**Problem 7.**
Run the code below to produce a list `families` consisting of inner lists, each with 3 items: the first two items are paired parents and the third is their child.

In [ ]:
# Mate each pair and store child in family next to its parents
families = []
for [p0, p1] in pairs: 
    child = mate_parents(p0, p1, d)
    families.append([p0, p1, child])
    
# Check our work
print(families)

**Problem 8.**
The function `improve_child` is defined in the code cell below. It applies the local improvement process to a child. Read through the code to get a sense of what it does.

In [ ]:
def improve_child(child, num_loc_improvement, n, d):
    """
    Locally improve child
    """
    # Local improvement on child's path p
    # 
    # 1. Randomly pick two vertices on path p: a and b
    #    - Vertex a should be before vertex b in path p
    # 2. Compare three candidate paths: 
    #    - original p, 
    #    - [s,...,a] on p + edge [a,b] + [b,...,t] on p, 
    #    - [s,...,a] on p + random vertex c + [b,...,t] on p, after removing loops. 
    # 3. Pick the best of these paths.
    # 4. Do this for num_loc_improvement iterations.
    
    for k in range(num_loc_improvement):
        child_path = child['path']
        
        a_index = np.random.randint(0, len(child_path) - 1)
        b_index = np.random.randint(a_index + 1, len(child_path))
        
        # Define and score first alternate path
        alt1_path = child_path[:a_index + 1] + child_path[b_index:]
        alt1_score = compute_score(alt1_path, d)
        alt1 = {'path': alt1_path, 'score': alt1_score}
        
        # Define and score second alternate path
        s = child_path[0]
        t = child_path[-1]
        a = child_path[a_index]
        b = child_path[b_index]

        # Choose a random vertex c, which cannot be s, t, a, or b
        found = False
        while not found:
            c = np.random.randint(0, n)
            if c not in [s, t, a, b]:
                found = True

        alt2_path = remove_loops(child_path[:a_index + 1] + [c] + child_path[b_index:])
        alt2_score = compute_score(alt2_path, d)
        alt2 = {'path': alt2_path, 'score': alt2_score}
        
        # Decide which path is best (lowest score)
        # Create list of candidates
        candidates = [child, alt1, alt2]
        
        # Initialize minimum score
        min_score = float("inf")
        
        # Initialize placeholder for candidate with minimum score
        min_cand = None
        
        for cand in candidates:
            if cand['score'] < float("inf"):
                min_score = cand['score']
                min_cand = cand

        # Update child to best path
        child = min_cand
        
    return child

The code below defines a child. Predict what will happen to the child after the first iteration of the `for` loop in `improve_child` above.

Then, run `improve_child` on `child`. Note that there are many random choices made by `improve_child`, and `improve_child` will perform many local improvements (`num_local_improvements = 100` from above).

In [ ]:
child_path = [0, 5, 8, 14, 16, 19, 23, 27, 29]
child_score = compute_score(child_path, d)
child = {'path': child_path, 'score': child_score}

# Write your code below


**Problem 9.**
Run the code below to produce a list `grown_families` of inner lists, each consisting of two paired parents and their improved "adult" child.

In [ ]:
# Locally improve each child and store result with parents
# in the list grown_families
grown_families = []
for [p0, p1, child] in families: 
    adult = improve_child(child, num_loc_improvement, n, d)
    grown_families.append([p0, p1, adult])

**Problem 10.**
The code cell below defines the function `keep_low_two`. What do you think this function does? 

In particular, what does the keyword argument `key=lambda member: member['score']` do for the `sorted` function? [Here's the documentation on `sorted`](https://docs.python.org/3.7/howto/sorting.html) - take a look at the "Key Functions" section.

In [ ]:
def keep_low_two(p0, p1, child):
    family = [p0, p1, child]
    sorted_family = sorted(family, key=lambda member: member['score'])
    
    return [sorted_family[0], sorted_family[1]]

# keep_low_two takes as input 3 individuals (dictionaries with paths and their 
# scores) and then returns a list of the two individuals with lowest scores.

Use the code below to rebuild the `parents` list, replacing some parents with their adult children, as appropriate. 

In [ ]:
# Rebuild list of parents, possibly using some children  
parents = []
for [p0, p1, adult] in grown_families:
    [new_p0, new_p1] = keep_low_two(p0, p1, adult)
    parents.append(new_p0)
    parents.append(new_p1)

**Problem 11.**
The code from Problems 5, 7, 9 and 10 runs a single generation of our genetic algorithm. Take all this code and put it inside a loop that runs the code `num_gens` times. Omit any print statements used to check your work; otherwise, your output will get messy fast!

**Problem 12.**
After running the code that we have so far, we should have a list of very fit parents, constructed and improved over multiple generations. 

The code cell below defines the function `get_fittest_parent` below. Read the function to get a sense of what it does.

In [ ]:
def get_fittest_parent(parents):
    """
    Gets the fittest parent from a list of parents
    """
    # Sort the parents, using their score as the key
    # We use the same technique as in Problem 10
    sorted_parents = sorted(parents, key=lambda parent: parent['score'])

    # Return parent with lowest
    return sorted_parents[0]

Write code that finds the parent with the lowest score and prints that parent's path and score to the screen.

In [ ]:
# Write your code below


**Problem 13.**
NetworkX has methods that efficiently solve shortest path problems. Note that the solution we obtained above using our genetic algorithm is heuristic (i.e., approximately optimal), while NetworkX's methods are exact (i.e. produces solutions that are guaranteed to be optimal). 

Let's compare our answer against the correct answer. Use the following code to solve the shortest path problem using NetworkX.  

In [ ]:
import networkx as nx

# Define complete graph
G = nx.complete_graph(n)

# Define distances for graph
for i in range(n):
    for j in range(n):
        if i != j:
            G[i][j]['weight'] = d[i][j]
        
# Solve shortest path problem
spath = nx.shortest_path(G, source=s, target=t, weight='weight')
spath_length = nx.shortest_path_length(G, source=s, target=t, weight='weight')

print(f'Actual shortest path: {spath}.')
print(f'The length of this path is : {spath_length}.')

**Problem 14.**
The code cell below contains a nicely organized version of the genetic algorithm that we've contructed above. Run this code or your own several times and see how often the genetic algorithm obtains an optimal solution. When it misses the correct solution, how badly does it miss? Report the error as 

$$
\text{error} = \text{genetic algorithm's score} - \text{actual minimum score}.
$$

What is the highest error that you observe?

In [ ]:
#############################################################
##### Import packages
#############################################################

import numpy as np
import networkx as nx

#############################################################
##### Set parameters
#############################################################

# Number of vertices
n = 30

# Source node
s = 0

# Terminal node
t = n - 1

# Number of parents
# Should be even
num_parents = 10

# Number of generations
num_gens = 50

# Number of local improvements
num_loc_improvement = 100

# Generate distance matrix
d = make_distance_matrix(n)

#############################################################
##### Define fitness function (compute_score)
#############################################################

def compute_score(path, d):
    """
    Compute the fitness score of a path, i.e., its total length
    """
    # Initialize score to 0
    score = 0
    
    # Iterate through the path nodes
    # Add the length of each edge to the score
    for e in range(len(path) - 1):
        score += d[path[e], path[e+1]]
        
    return score

#############################################################
##### Randomly generate a list of parents
#############################################################

# Each parent is a dictionary containing a path and its score.
parents = generate_parents(num_parents, s, t, n, d)

#############################################################
##### Enter the main loop: loop over generations
#############################################################

for gen in range(num_gens): 
               
    #############################################################
    ##### Produce children
    #############################################################
        
    # P5: Pair parents for mating
    random_index = np.random.permutation(list(range(num_parents)))
    pairs = []
    for i in range(int(num_parents / 2)):
        pairs.append([parents[random_index[2 * i]], parents[random_index[2 * i + 1]]])
    
    # P7: Mate each pair and store child
    families = []
    for [p0, p1] in pairs: 
        child = mate_parents(p0, p1, d)
        families.append([p0, p1, child])
        
    #############################################################
    ##### Locally improve children
    #############################################################
    
    # P9: Locally improve each child
    grown_families = []
    for [p0, p1, child] in families: 
        adult = improve_child(child, num_loc_improvement, n, d)
        grown_families.append([p0, p1, adult])
   
    #############################################################
    ##### Each child tries to replace one of their parents
    #############################################################
    
    # P10: Rebuild list of parents, possibly using some children  
    parents = []
    for [p0, p1, adult] in grown_families:
        [new_p0, new_p1] = keep_low_two(p0, p1, adult)
        parents.append(new_p0)
        parents.append(new_p1)

#############################################################
##### End main loop
#############################################################

#############################################################
##### Take fittest parent as our solution
#############################################################

best_parent = get_fittest_parent(parents)
print(f"The best path found is: {best_parent['path']}.")
print(f"The length of this path is {best_parent['score']}.")

#############################################################
##### Compare against the answer from NetworkX
#############################################################

# Define complete graph
G = nx.complete_graph(n)

# Define distances for graph
for i in range(n):
    for j in range(n):
        if i != j:
            G[i][j]['weight'] = d[i][j]
        
# Solve shortest path problem
spath = nx.shortest_path(G, source=s, target=t, weight='weight')
spath_length = nx.shortest_path_length(G, source=s, target=t, weight='weight')

print(f'Actual shortest path: {spath}.')
print(f'The length of this path is : {spath_length}.')

#############################################################
##### Write your code below: what is the error?
#############################################################

print(f"\nThe error is {best_parent['score'] - spath_length}.")

**Problem 15.**
There is a lot that is unknown about genetic algorithms. For example: How should the various parameters be set? Which local and mating processes should be used? 

The answers to these questions are typically different for different problems. Can you find better parameter values to use for this problem? One might expect that using more generations or more local modifications or more parents will be better. However, limit yourself: adjust these parameters so that

```
num_parents * num_gens * num_loc_improvement <= 50000
```

**Problem 16. (Variant of the Shortest Path Problem)**
A variant of the shortest path problem involves a traveler planning a trip from city $s$ to city $t$. Here the vertices of the graph represent cities and the values in $d_{ij}$ represent distances. However, the traveler finds it difficult to leave each wonderful city he visits in his journey; so difficult that we multiply the $d_{ij}$ by 2 each time that he leaves a city. For instance, if $d_{01} = 5$, $d_{12} = 3$, and $d_{23} = 6$ then the $0$-$3$ path $0$-$1$-$2$-$3$ has score $5 + 2(3) + (2^2)6 = 35$. 

This problem cannot be solved using the standard shortest path algorithms (e.g. Dijkstra, Bellman-Ford), but it is fairly easy to modify your genetic algorithm above to find a heuristic solution. All you need to do is to modify your `compute_score` function! 

Make the correct modification and solve the problem for several randomly generated examples. Can you find an instance where the genetic algorithm's solution is better than the path proposed by NetworkX's `shortest_path` method (using the usual $d$ matrix for the distances), and the length of the path found is computed manually according to the new scoring rule for the above traveler?